[**requests** Docs](https://requests.readthedocs.io/en/latest/index.html)

[**BeautifulSoup** Docs](https://beautiful-soup-4.readthedocs.io/en/latest/)

[**wget** Docs](https://pypi.org/project/wget/)


___
## <center><a id=1>Парсинг сайта</a>

### `Поиск ссылки`

In [155]:
import requests 
from bs4 import BeautifulSoup
import re


site_url = 'https://www.ncbi.nlm.nih.gov'  # Url сайта
datasets_url = '/geo/query/acc.cgi'        # Url поиска датасетов
parameter = 'GSE68849'                     # Название датасета

response = requests.get(site_url + datasets_url, params={'acc' : parameter})  # GET-запрос к поиску конкретного датасета
page = BeautifulSoup(response.text, 'html.parser')                            # Считывание html-информации

attribute = 'acc=' + parameter                                                # Атрибут по которому фильтруются ссылки

links = page.find_all(name='a', href=[ re.compile(attribute)])                # Поиск списка ссылок с параметром - название датасета

for link in links:
    search_link = re.compile(r'format=file$').search(link.get('href'))        # Поиск ссылки из списка с форматом только файл
    if search_link is not None:
        result_search = link
result_search = result_search.get('href')                                     # Извлечение текста относительной ссылки
download_link = site_url + result_search                                      # Получение ссылки для скачивания датасета
download_link

'https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE68849&format=file'

### `Создание папки`

In [ ]:
import os

try:
    os.mkdir('data')
except FileExistsError:
    print('_'*31, "\nThis directory already exists !\n",'_'*30)

### `Скачивание файла`

In [153]:
import wget

wget.download(download_link, out='data/')  #  Без "/" назначается новое название файла

'data//GSE68849_RAW.tar'

___
## <center><a id=2>Распаковка архива</a>

[**os** Docs](https://docs.python.org/3/library/os.html)

[**tarfile** Docs](https://docs.python.org/3/library/tarfile.html#module-tarfile)
| [**tarfile** About](https://docs-python.ru/standart-library/modul-tarfile-python/metody-obekta-tarfile-modulja-tarfile/#tarfile.TarFile.open)

[**gzip** Docs](https://docs.python.org/3/library/gzip.html#module-gzip)
| [**gzip** About](https://docs-python.ru/standart-library/modul-gzip-python/)

In [2]:
import os
import tarfile
import gzip
import shutil

def extract_file(filedir_path:str) -> int:
    
    """Распаковывает архив в текущей папке

    Args:
        filedir_path (str): Путь к папке с архивом

    Returns:
        int: Если архив распакован, возвращает 1, если не распакован - 0
    """
    os.chdir(filedir_path)
    extracted_flag = 0
    filenames_list = list(os.walk(filedir_path))[0][2]
    for filename in filenames_list:
        filename_short = os.path.splitext(filename)[0]
        file_extension = os.path.splitext(filename)[1]
        
        if file_extension in ['.tar', '.gzip', '.bz2', '.lzma'] :    
            with tarfile.open(filename) as tar:
                tar.extractall()
            extracted_flag = 1
            os.remove(filename)
                
        if file_extension in ['.gz'] :
            with gzip.open(filename, 'rb') as file_in:
                with open(filename_short, 'wb') as file_out:
                    shutil.copyfileobj(file_in, file_out)
            extracted_flag = 1
            os.remove(filename)
    return extracted_flag

In [ ]:
filedir_path = 'd:\\Learning\\Yandex Disk Sync\\YandexDisk\\MIPT.Data Science\\Инжиниринг данных\\Luigi\\Luigi Practice\\data'

extracted_flag = 1
while extracted_flag == 1:

    extracted_flag = extract_file(filedir_path)
    
extracted_flag

___
## <center><a id=3>Создание структуры</a>

In [37]:
import os

def replace_files(filedir_path:str) -> None:
    """Перемещает файлы в созданные папки 

    Args:
        filedir_path (str): Путь к папке с файлами для перемещения
    """
    os.chdir(filedir_path)
    filenames_list = list(os.walk(filedir_path))[0][2]
    for filename in filenames_list:
        filename_short = os.path.splitext(filename)[0]
        try:
            os.mkdir(filename_short)
        except FileExistsError:
            print('_'*31, "\nThis directory already exists !\n",'_'*30)
        os.replace(filename, filename_short +'\\' + filename)

In [38]:
filedir_path = 'd:\\Learning\\Yandex Disk Sync\\YandexDisk\\MIPT.Data Science\\Инжиниринг данных\\Luigi\\Luigi Practice\\data'
replace_files(filedir_path)

_______________________________ 
This directory already exists !
 ______________________________
_______________________________ 
This directory already exists !
 ______________________________


___
## <center><a id=4>Экспорт  из csv в txt </a>

In [4]:
import io
import pandas as pd
import os

def export_to_csv(filename:str) -> None:
    dfs = {}
    with open(filename) as f:
        write_key = None
        fio = io.StringIO()
        for l in f.readlines():
            if l.startswith('['):
                if write_key:
                    fio.seek(0)
                    header = None if write_key == 'Heading' else 'infer'
                    dfs[write_key] = pd.read_csv(fio, sep='\t', header=header)
                fio = io.StringIO()
                write_key = l.strip('[]\n')
                continue
            if write_key:
                fio.write(l)
        fio.seek(0)
        dfs[write_key] = pd.read_csv(fio, sep='\t')
    for df_name in dfs:
        df = pd.DataFrame(dfs[df_name])
        df.to_csv(f'{df_name}.csv', index=False)


In [5]:
filedir_path = 'd:\\Learning\\Yandex Disk Sync\\YandexDisk\\MIPT.Data Science\\Инжиниринг данных\\Luigi\\Luigi Practice\\data'

def export_to_dir(filedir_path:str) -> None:
    dirs_list = list(os.walk(filedir_path))[0][1]
    for dir in dirs_list:
        new_dir_path = filedir_path + '\\' + dir
        os.chdir(new_dir_path)
        filenames_list = list(os.walk(new_dir_path))[0][2]
        for filename in filenames_list:
            export_to_csv(filename)

In [6]:
export_to_dir(filedir_path)

___
## <center><a id=5>Редактирование файлов </a>

In [ ]:
import pandas as pd
import os


filedir_path = os.getcwd()
filename = 'Probes.csv'

dirnames_list = list(os.walk(filedir_path))[0][1]

for dirname in dirnames_list:
    print(dirname)
    
    os.chdir(filedir_path + '\\' + dirname)
    df = pd.read_csv(filename)
    df_edited = df.drop(columns=['Definition', 'Ontology_Component', 'Ontology_Process', 'Ontology_Function', 'Synonyms', 'Obsolete_Probe_Id', 'Probe_Sequence'])
    df_edited.to_csv('Probes_short.csv', index=False)
    os.chdir(filedir_path)
    print(df_edited.info())

___
___
___
___